In [20]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch

from torchvision.models import ResNet18_Weights, resnet18
from tqdm.notebook import tqdm_notebook
from torchvision import datasets, transforms

from src.imagenet_x.evaluate import ImageNetX, get_vanilla_transform, ImageNetXImageFolder, load_annotations
from src.imagenet_x import FACTORS, plots

In [7]:
imagenet_val_path = '../data/ImageNetVal'
transforms = get_vanilla_transform()
dataset = ImageNetX(imagenet_val_path, which_factor='top', partition='val', transform=transforms, filter_prototypes=False)    

In [18]:
class BachelorDataFolder(datasets.ImageFolder):
    
    def __init__(self, folder_path, *args, **kwargs):
        super().__init__(folder_path, *args, **kwargs)


BachelorDataFolder(imagenet_val_path)


Dataset BachelorDataFolder
    Number of datapoints: 50200
    Root location: ../data/ImageNetVal

In [147]:
#rows with errors:
#17758
#29800 
df = load_annotations(filter_prototypes=False)
df = df.drop([17758, 29800]).reset_index(drop=True)
df_filenames = pd.DataFrame(df['file_name'])
df3 = df.drop(columns=['class', 'justification', 'one_word', 'metaclass'])

In [148]:
df_filenames.shape

(48866, 1)

In [149]:
df4 = pd.from_dummies(df3.drop(columns=['file_name']))

In [160]:
df5 = df_filenames.join(df4)

In [161]:
df5.columns = ['file_name','class']

In [166]:
df5.shape

(48866, 2)

In [167]:
df5

,file_name,class
0,ILSVRC2012_val_00004487.JPEG,larger
1,ILSVRC2012_val_00003963.JPEG,color
2,ILSVRC2012_val_00041992.JPEG,pose
3,ILSVRC2012_val_00028056.JPEG,pattern
4,ILSVRC2012_val_00016832.JPEG,darker
...,...,...
48861,ILSVRC2012_val_00004973.JPEG,color
48862,ILSVRC2012_val_00038750.JPEG,background
48863,ILSVRC2012_val_00043497.JPEG,color
48864,ILSVRC2012_val_00005859.JPEG,pattern
